In [1]:
from astropy.io import fits
import numpy as np
import os.path
import matplotlib.pyplot as plt
from astropy import wcs
from astropy.nddata import Cutout2D
from astropy import coordinates
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import matplotlib.gridspec as gridspec
import aplpy
import pandas as pd
%matplotlib inline
from photutils import CircularAperture
from photutils import SkyCircularAperture
from photutils import aperture_photometry

In [2]:
a = ['goods_b', 'goods_v', 'goods_i', 'goods_z']
b = ['goods_b', 'goods_v', 'goods_i', 'goods_z']
c = ['goods', 'goods', 'goods', 'goods']
path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/big_table.fits'
big_table = fits.open(path)[1].data

for i in range(len(big_table)):
    if (big_table.field('Visual_Flag')[i] > 3):
        im_b = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_b' + str(i+1) + '.fits'
        im_v = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_v' + str(i+1) + '.fits'
        im_i = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_i' + str(i+1) + '.fits'
        im_z = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_z' + str(i+1) + '.fits'
        wht_b = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_b' + str(i+1) + 'wht.fits'
        wht_v = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_v' + str(i+1) + 'wht.fits'
        wht_i = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_i' + str(i+1) + 'wht.fits'
        wht_z = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_z' + str(i+1) + 'wht.fits'
        conf = '/Users/justin/Documents/Master_Thesis/data/catalogues/goods/goods_stack.swarp'
        im_1 = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods_stacks/cutout_goods_bv' + str(i+1) + '.fits'
        im_2 = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods_stacks/cutout_goods_bvi' + str(i+1) + '.fits'
        im_3 = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods_stacks/cutout_goods_bviz' + str(i+1) + '.fits'
        wht_1 = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods_stacks/cutout_goods_bv' + str(i+1) + 'wht.fits'
        wht_2 = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods_stacks/cutout_goods_bvi' + str(i+1) + 'wht.fits'
        wht_3 = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods_stacks/cutout_goods_bviz' + str(i+1) + 'wht.fits'
        swarp1 = 'swarp ' + im_b + ' ' + im_v + ' -c ' + conf + ' -IMAGEOUT_NAME ' + im_1 + ' -WEIGHTOUT_NAME ' + wht_1 + ' -WEIGHT_IMAGE ' + wht_b + ',' + wht_v
        swarp2 = 'swarp ' + im_b + ' ' + im_v + ' ' + im_i + ' -c ' + conf + ' -IMAGEOUT_NAME ' + im_2 + ' -WEIGHTOUT_NAME ' + wht_2 + ' -WEIGHT_IMAGE ' + wht_b + ',' + wht_v + ',' + wht_i
        swarp3 = 'swarp ' + im_b + ' ' + im_v + ' ' + im_i + ' ' + im_z + ' -c ' + conf + ' -IMAGEOUT_NAME ' + im_3 + ' -WEIGHTOUT_NAME ' + wht_3 + ' -WEIGHT_IMAGE ' + wht_b + ',' + wht_v + ',' + wht_i + ',' + wht_z
        os.system(swarp1)
        os.system(swarp2)
        os.system(swarp3)
        


In [3]:
d=['bv','bvi','bviz']
n_thresh=64
det_thr=0.5
min_cont=0.0001
aper_pixel=15.
min_area=20.
for i in range(len(big_table)):
    if (big_table.field('Visual_Flag')[i] > 3):
        min_area=big_table.field('Min_Area')[i]
        aper_pixel=big_table.field('Aperture')[i] / 0.03
        for j in range(len(d)):
            im = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods_stacks/cutout_goods_' + d[j] + str(i+1) + '.fits'
            wht = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods_stacks/cutout_goods_' + d[j] + str(i+1) + 'wht.fits'
            conf = '/Users/justin/Documents/Master_Thesis/data/catalogues/goods/goods_b.conf'        
            cat = '/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/goods_stacks/cat_goods_' + d[j] + str(i+1) + '.cat'
            check = '/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/goods_stacks/cat_goods_' + d[j] + str(i+1) + '_aper.fits,/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/goods_stacks/cat_goods_' + d[j] + str(i+1) + '_bkg.fits,/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/goods_stacks/cat_goods_' + d[j] + str(i+1) + '_bkg_rms.fits' 
            param = '/Users/justin/Documents/Master_Thesis/data/catalogues/goods/goods_stacks.param'
            sex = 'sex -c ' + conf + ' ' + im + ' -CATALOG_NAME ' + cat + ' -WEIGHT_TYPE MAP_RMS -WEIGHT_GAIN N -WEIGHT_IMAGE ' + wht + ' -DETECT_MINAREA ' + str(min_area) + ' -PHOT_APERTURES ' + str(aper_pixel) + ' -CHECKIMAGE_NAME ' + check + ' -BACK_SIZE 128 -DEBLEND_MINCONT ' + str(min_cont) + ' -DEBLEND_NTHRESH ' + str(n_thresh) + ' -MASK_TYPE NONE -DETECT_THRESH ' + str(det_thr) + ' -ANALYSIS_THRESH ' + str(det_thr) + ' -PARAMETERS_NAME ' + param     
            os.system(sex)